In [1]:
import os
import sys
import atoti as tt
import pandas as pd

In [2]:
session = tt.Session(
    # port=9098,
    logging=tt.LoggingConfig(destination="./logging/outer_join.log")
)

In [3]:
benchmark = session.read_csv("./sample_data/benchmark.csv")

In [4]:
benchmark.head()

,AsOfDate,PortfolioTicker,BenchmarkTicker,CUSIP,BenchmarkMarketValue
0,1/31/24,GBTLDQ,BMMLC04,US0000001,1000
1,1/31/24,GBTLDQ,BMMLC04,GB0000002,4000
2,1/31/24,GBTLDQ,BMMLC04,US0000010,5000
3,1/31/24,GBTLDQ,BMMLC04,US0000011,2000
4,1/31/24,GBTLDQ,BMMLC04,US0000012,3000


In [5]:
holding = session.read_csv("./sample_data/holding.csv")

In [6]:
holding.head()

,AsOfDate,PortfolioTicker,CUSIP,HoldingMarketValue
0,1/31/24,GBTLDQ,US0000001,1000
1,1/31/24,GBTLDQ,US0000002,2000
2,1/31/24,GBTLDQ,US0000004,6000
3,1/31/24,GBTLDQ,US0000003,4000


In [7]:
table = session.create_table(
    name="base",
    types={
        "AsOfDate": tt.LOCAL_DATE,
        "mv": tt.DOUBLE,
        "CUSIP": tt.STRING,
        "PortfolioTicker": tt.STRING,
        "BenchmarkTicker": tt.STRING,
    },
    keys=["AsOfDate","CUSIP", "PortfolioTicker", "BenchmarkTicker"],
    default_values={
        "PortfolioTicker": "fake_ben",
        "BenchmarkTicker": "fake_por"
    }
)

In [8]:
table.load_csv("./sample_data/benchmark.csv")
table.load_csv("./sample_data/holding.csv")

6 error(s) occurred while feeding the table base. Check the session's logs for more details.
4 error(s) occurred while feeding the table base. Check the session's logs for more details.


In [9]:
benchmark_members=pd.read_csv("./sample_data/benchmark.csv")['BenchmarkTicker'].unique().tolist()
portfolio_members=pd.read_csv("./sample_data/holding.csv")['PortfolioTicker'].unique().tolist()

In [10]:
cube = session.create_cube(holding)
m, l, h = cube.measures, cube.levels, cube.hierarchies

In [11]:
cube.create_parameter_hierarchy_from_members(name="Benchmark_id", members=benchmark_members)

In [12]:
cube.create_parameter_hierarchy_from_members(name="_portfolio_id", members=["fake_ben"])

In [11]:
h

{('Holding', 'CUSIP'): <atoti.hierarchy.Hierarchy object at 0x13a462e40>, ('Holding', 'PortfolioTicker'): <atoti.hierarchy.Hierarchy object at 0x13a462180>, ('Holding', 'AsOfDate'): <atoti.hierarchy.Hierarchy object at 0x13a460260>}

In [13]:
h["_portfolio_id"].visible=False
h["BenchmarkTicker"].visible=False

KeyError: 'BenchmarkTicker'

In [ ]:
h["_portfolio_id"].slicing=True
h["PortfolioTicker"].slicing=True

In [ ]:
m["portfolio_mv"] = tt.filter(m["mv.SUM"], l["benchmark_id"] == "fake_por")

In [ ]:
session.widget